# PySub Tutorial 4 - Advanced plotting

Welcome to the fourth example case for building! We are going to use the previously determined subsidence model from Tutorial 1 to display the resulting data in various ways which might be common when making figures.

This tutorial can be found in the folder "Tutorials", in the folder where your PySub package has been installed.

The case we are studying is a the result of Tutorial 1, so make sure you have run that code so the model file is saved in the Tutorial 1 folder. The goal is to familiarize you with the PySub model, it's syntax and functionality. 

The PySub modules used in this tutorial are: plot_utils, Geometries and memory

In this tutorial we will show you how to:
- Adjust the plots to your liking:
    - Change the colors of the contours
    - Change the title
    - Add a new legend
    - Reuse of the figure for not implemented applications
- Add new data to the model so it can also be plotted with the PySub plot functions
- Adjust how the reservoirs are plotted in the figures
- Add additional shapes to the figure


## The code
In below cells, an example is given of PySub code where the subsidence is loaded.

After this, methods to refine your plots will be discussed.

In [ ]:
from PySub.memory import load
from PySub import plot_utils as plot
load_file = r'Tutorial 1\save\Tutorial 1.smf'
Model = load(load_file)

plot.plot_subsidence(Model)

## Background layers
The standard background maps chosen in PySub are a ArcGIS set of topographical maps. Other maps can be added by finding the link to its wmts service and its layer to the arguments of the plot function. Chances are, that the WMTS function is not in the same coordinate system that your data is in, so specify the coordinate system your data is in.

In [ ]:
help(plot.set_background)

In [ ]:
plot.set_background(google_service = 'only_streets')
plot.plot_subsidence(Model)

You can reset to the default using:

In [ ]:
plot.set_background(arcgis_service = 'World_Topo_Map')

# Adjusting visuals
The PySub plotting functions pass arguments to matplotlib.pyplot functions to dictate the plotted result. The default arguments are stored in the Model object. If we want to see the default values we can go to the model object and check which defaults values are set for each type of function. 

In [ ]:
Model.contourf_defaults

The defaults can be set with:

In [ ]:
Model.set_contourf_defaults({'cmap': 'jet'})

The explanation on how to adjust the defaults and which are available can be found by passing the desired setting function to the help function:

In [ ]:
help(Model.set_contourf_defaults)

When setting the default kwargs for that function all the plotting functions that make filled contours will use these arguments. If you want a specific call to a function you can set the keyword arguments in the function as well:

In [ ]:
cmap = 'gist_rainbow_r'
colormap_kwargs = {'contourf_kwargs': {'cmap': cmap},
                   'contour_kwargs': {'cmap': cmap}}
plot.plot_subsidence(Model, **colormap_kwargs)

The possible kwargs are visible in the help function for the plot function.

## Shapes
The reservoirs are shown in the figures as polygons (standard green). Caverns would be displayed as points. It is also possible to have grids shown as a colored surface. Each of these type of spatial representations of a reservoir or cavern are stored in a geometry object from PySub.

In this part, we are discussing the use of shapes in the PySub tool. It is not the aim of the PySub tool to make editable shapes. We recommend GIS software to make figures using poylgons, lines and points. PySub's tools for this are limited.

The shapes of the reservoirs can be removed from the final figure by setting the plot_reservoir_shapes argument to False:

In [ ]:
plot.plot_subsidence(Model, plot_reservoir_shapes = False)

Note that the contours are now plotted with the "jet" colormap as set above with the set_contourf function. 

## Highlighting a shape

The shapes can be highlighted by setting the specific color of a shape. There are 3 reservoirs that have polygons plotted for the gas reservoirs. For shapes that are a different type of geometry, these should be considered in the entry of the color arguments as raster_kwargs or scatter_kwargs.

In [ ]:
yellow         = (1, 1, 0) # RGB, RGBA or as string: 'yellow'
green          = (0, 1, 0) # RGB, RGBA or as string:  'green'
gas_reservoirs = ['Norg', 'Allardsoog', 'Een']
colors         = [yellow,  green,        green]

for color, reservoir in zip(colors, gas_reservoirs): 
    print(reservoir, color)

Now that we have determined the colors per reservoir, we can add them to the facecolor (or fc) argument. To show of some other variables which can be adjusted:

- the linestyle (ls) is set to be dotted
- the edgecolor (ec) is set to red
- the linewidth (lw) is set to 3 in points

For more adjusteable parameters look [here](https://matplotlib.org/stable/api/_as_gen/matplotlib.patches.PathPatch.html). 

In [ ]:
plot.plot_reservoirs(Model, shape_kwargs = {'fc': colors, 'ls': ':', 'ec': 'r', 'lw': 3})

## Adding a different shape
The plot_subsidence function automatically uses the shapes that were used as input to the model. But maybe you want to compare it with another shape. For this, we can use the additional_shapes parameter of the plot subsidence function and the fetch method from the Geometries module. The fetch function takes the location or xy coordinates of a point or polygon as input.

Below, the shapes for the original input reservoirs are fetched, and an additional shape for Zevenhuizen-West. The original shapes used in the model are replaced with the additional files by setting the plot_reservoir_shapes to False. This way you can use different shapes for the reservoirs when plotting.

In [ ]:
from PySub.Geometries import fetch

additional_shapes = fetch([
    r'Shapefiles/Norg.shp',
    r'Shapefiles/Allardsoog.shp',
    r'Shapefiles/Een.shp',
    r'Shapefiles/Zevenhuizen-West.shp',
])

plot.plot_reservoirs(
    Model, 
    plot_reservoir_shapes = False, 
    additional_shapes = additional_shapes)

Below figure has had some additional editing done:
- The background map would not show enough detail with this level of zoom, so we set the zoom_level to 12 (default is 10).
I want to add some additional information to this figure, not native to PySub! Like:
- a legend for the colors. We want it clear that the yellow field is the field that's going to be producing and the others have been producing already. Therefore, We want the labels to say yellow = "Producing soon" and green = "Producing".
- labels for the x- and y-axis
- I want top adjust the extent of the axis beyond the zoom.
- Then, I want to save the figure in the output folder

We can retreive the figure for additional plotting by setting final to False. This returns a matplotlib fig and ax object, allowing for further manipulation of the figure. The fig and ax objects can be adjusted using [matplotlib](https://matplotlib.org/) methods.

When final is set to False, the figure will not save or plot automatically. The matplotlib.pyplot module is used to plot (with plt.show()) and to save (with plt.savefig).

In [ ]:
import os
import matplotlib.pyplot as plt 

# Return a fig and ax object
fig, ax = plot.plot_reservoirs(
    Model, 
    shape_kwargs = {'fc': colors}, 
    final = False,
    buffer = -5000, # In m
    zoom_level = 12,
)

# Set the axis labels
ax.set_xlabel('RD (m)')
ax.set_ylabel('RD (m)')
ax.set_xlim(215000, 230000)
ax.set_ylim(565000, 572000)


# We only want the color of the shapes to change between polygons so we copy the default values.
# For more info, use help(plot.add_custom_legend).
legend_kwargs = [Model.shape_defaults.copy() for c in (yellow, green)]
for i, c in enumerate((yellow, green)):
    legend_kwargs[i]['facecolor'] = c

# Make some good labels:
legend_labels = ['Producing soon', 'Producing',]
    
plot.add_custom_legend(ax, 'polygon', 
                       kwargs = legend_kwargs, 
                       labels = legend_labels)


path_to_new_figure = Model.project_folder.output_file("modified figure")
plt.savefig(path_to_new_figure, dpi = 'figure', bbox_inches = 'tight')
plt.show()

# Displaying new data
By setting values to the SubsidenceModel object (that are xarray Datasets or DataArrays) we can display new data. When at least the data is available in the x- and y-dimension we can show it in 2D. In below cell, an example is made by setting a variable that is the difference between two instances in time:

In [ ]:
Model.subsidence_2000_2010 = (
    Model.subsidence.sel(time = '1-1-2010') - 
    Model.subsidence.sel(time = '1-1-2000')
)
plot.plot_subsidence(
    Model, 
    variable = "subsidence_2000_2010", 
    contour_steps = 0.001,
    title = "Subsidence between 2000 and 2010 (cm)"
)

Note that the title has been changed by setting the title variable.

# Plotting points on a map
You can add points on the map in many ways. See variable points here:

In [ ]:
help(plot.plot_points_on_map)

These can be your model.observation_points or your model.points

In [ ]:
plot.plot_points_on_map(Model, Model.points)

# Selecting points
You can make your own points using the Points module. This part makes liberal use of "[list comprehensions](https://stackoverflow.com/a/68408462)", to make the code much shorter.

In [ ]:
help(Points.PointCollection)

In the cell below we use the extent of the plot and the line x = 217500 to clip the points. As you can see in the plot, the point east of that line is omitted from the plot.

In [ ]:
from PySub import Points
selection_x = 205000, 217500
selection_y = 555000, 580000

def isin_selection(point, selection_x, selection_y):
    # Returns True when the point is in selection, or False when not.
    return (selection_x[0] < point.x < selection_x[1]) & (selection_y[0] < point.y < selection_y[1])


names_of_points_in_selection = [
    p.name for p in Model.points if isin_selection(p, selection_x, selection_y)
]

points = Points.PointCollection(
    Model.points[names_of_points_in_selection]
)

plot.plot_points_on_map(Model, points=points)

# Plotting observations on a map
Plotting observations uses the same function, but requires a different argument for the plots. Instead of a Points.PointCollection object, you can also select the Point.ObservationCollection object.

In [ ]:
help(Points.ObservationCollection)

Using this functionality we make the same selection as before, using the line x = 217500 to clip the observations.

In [ ]:
names_of_observations_in_selection = [
    p.name for p in Model.observation_points if isin_selection(p, selection_x, selection_y)
]

observations = Points.ObservationCollection(
    Model.observation_points[names_of_observations_in_selection]
)
plot.plot_points_on_map(Model, points=observations, scatter_kwargs = {"marker": "^", "fc": "r", "ec":"k"})

The most flexible way to plot and label points is using lists (or list-like objects like numpy arrays) as the points argument. To have labels assigned to these, you can add the labels argument. Labels must have the same length the amount of points, but not all values need to be text. Enter "" to not add a label.

In [ ]:
point_1 = [215000, 570345]
point_2 = [214852, 565000]
point_3 = [226546, 585000] # This point is far away from the subsidence bowl. We are not interested in it.
plot.plot_points_on_map(
    Model, 
    points=[point_1, point_2, point_3], 
    labels = ["point 1", "point 2", ""], 
    scatter_kwargs = {"marker": "^", "fc": "r", "ec":"k"}
)

An analog with a Points object is described below. here too all points are plotted, but only a selection has a label.
This cell performs selection based on coordinates:

In [ ]:
labels = [
    p.name 
    if isin_selection(p, selection_x, selection_y) else "" 
    for p in Model.observation_points
]
plot.plot_points_on_map(
    Model, 
    points=Model.observation_points, 
    labels=labels,
    scatter_kwargs = {"marker": "^", "fc": "r", "ec":"k"}
)

And this cell based on a selection of possibly multiple points of interest:

In [ ]:
observations_of_interest = ["00000001"]

labels = [
    p.name 
    if p.name in observations_of_interest else "" 
    for p in Model.observation_points
]
plot.plot_points_on_map(
    Model, 
    points=Model.observation_points, 
    labels=labels,
    scatter_kwargs = {"marker": "^", "fc": "r", "ec":"k"}
)